In [34]:
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
from langchain import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage


import os
import openai


openai.api_type = "azure"
openai.api_base = "https://open-ai-maspex.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()
# initialize the models
model = AzureOpenAI(
        deployment_name="davinci",
        model_name='text-davinci-003',
        temperature=1
    )
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')


In [23]:
load_dotenv()
file_path = './data/book4.pdf'
loader = PyPDFLoader(file_path)
text = loader.load()

In [24]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(text)
db = FAISS.from_documents(docs, embeddings)
vector_store = db.save_local('faiss_index')

In [38]:
new_db = FAISS.load_local('faiss_index', embeddings)

In [39]:
def make_chain():
    return ConversationalRetrievalChain.from_llm(
        model, 
        new_db.as_retriever(),
        return_source_documents=True)

if __name__ == '__main__':
    load_dotenv()

    chain = make_chain()
    chat_history = []

    while True:
        print()
        question = input("Enter a question: ")

        response = chain({"question": question, "chat_history": chat_history})

        answer = response["answer"]
        source = response["source_documents"]
        chat_history.append(HumanMessage(content=question))
        chat_history.append(AIMessage(content=answer))

        print("/n/nSources:\n")
        for document in source:
            print(f"Page: {document.metadata['page']}")
            print(f"Text: {document.page_content}...\n")
        print(f"Answer: {answer}")
    


/n/nSources:

Page: 312
Text: 9.7. PRACTICE 297
No. Those problems were probably always there, even in the other tools. But since Gibbs
doesn’t use gradients, it doesn’t notice some issues that a Hamiltonian engine will. A culture
has evolved in applied statistics of just running bad chains for a very long time—for millions of
iterations—and then thinning aggressively, praying, and publishing. This must stop. [example of
DOI:https://doi.org/10.1016/j.cub.2018.03.020 — 5 million samples, neff of 66!] Tools like Stan and
other Hamiltonian engines are so important for reliable research precisely because they more diag-
nostic criteria for the accuracy of the Monte Carlo approximation. Don’t resent the nagging.
9.6. Summary
This chapter has been an informal introduction to Markov chain Monte Carlo (MCMC)
estimation. The goal has been to introduce the purpose and approach MCMC algorithms.
The major algorithms introduced were the Metropolis, Gibbs sampling, and Hamiltonian
Monte Carlo algor

KeyboardInterrupt: 